In [0]:
! pip install psutil
import psutil, time
if psutil.virtual_memory().total < 2e10:
  lis = []
  i = '0' * 100000000  # 100mb
  while 1:
    if psutil.virtual_memory().free < 3e8:
      time.sleep(1)
    i += '0'
    lis.append(i)

In [0]:
! wget https://raw.githubusercontent.com/HCIT-Computing-Intelligence/datasource/master/export_sql_1558435.zip

--2020-03-18 03:45:33--  https://raw.githubusercontent.com/HCIT-Computing-Intelligence/datasource/master/export_sql_1558435.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42396032 (40M) [application/zip]
Saving to: ‘export_sql_1558435.zip.1’

export_sql_1558435. 100%[===================>]  40.43M   147MB/s    in 0.3s    

2020-03-18 03:45:33 (147 MB/s) - ‘export_sql_1558435.zip.1’ saved [42396032/42396032]



In [0]:
! unzip export_sql_1558435.zip

Archive:  export_sql_1558435.zip
replace sqlResult_1558435.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
! pip install pyltp

In [0]:
! wget http://model.scir.yunfutech.com/model/ltp_data_v3.4.0.zip

In [0]:
! unzip ltp_data_v3.4.0.zip

In [0]:
! pip install -U gensim

In [0]:
! wget https://ai.tencent.com/ailab/nlp/data/Tencent_AILab_ChineseEmbedding.tar.gz

In [0]:
! tar -zxvf Tencent_AILab_ChineseEmbedding.tar.gz

In [0]:
import pandas as pd
import numpy as np
import time, os, json, pickle, pyltp
from pyltp import SentenceSplitter,NamedEntityRecognizer,Postagger,Parser,Segmentor

In [0]:
cws_model = "ltp_data_v3.4.0/cws.model"
pos_model = "ltp_data_v3.4.0/pos.model"
par_model = "ltp_data_v3.4.0/parser.model"
ner_model = "ltp_data_v3.4.0/ner.model"


def get_word_list(sentence,model=cws_model):
    #得到分词
    segmentor = Segmentor()
    segmentor.load(model)
    word_list = list(segmentor.segment(sentence))
    segmentor.release()
    return word_list

def get_postag_list(word_list,model=pos_model):
    #得到词性标注
    postag = Postagger()
    postag.load(model)
    postag_list = list(postag.postag(word_list))
    postag.release()
    return postag_list

def get_parser_list(word_list,postag_list,model=par_model):
    #得到依存关系
    parser = Parser()
    parser.load(model)
    arcs = parser.parse(word_list,postag_list)
    arc_list = [(arc.head - 1,arc.relation) for arc in arcs]
    parser.release()
    return arc_list

In [0]:
from gensim.models import KeyedVectors
wv_from_text = KeyedVectors.load_word2vec_format('Tencent_AILab_ChineseEmbedding.txt', binary=False)

In [0]:
wv = wv_from_text
wv['今天']

array([ 0.279403, -0.16954 , -0.255177,  0.347896,  0.451785, -0.205646,
        0.191702,  0.330589, -0.037373,  0.221849,  0.054098,  0.024398,
       -0.182197, -0.009881,  0.083631, -0.029917, -0.141319, -0.07029 ,
       -0.104024, -0.28636 ,  0.076999,  0.145647,  0.203121,  0.20046 ,
        0.32983 , -0.260944, -0.341079, -0.088483,  0.308596,  0.025227,
        0.199181,  0.176566, -0.182045,  0.134017,  0.040898,  0.065718,
        0.137861,  0.293695, -0.381076, -0.123865,  0.195901, -0.141544,
        0.545666,  0.099889, -0.039392, -0.264894, -0.328588, -0.518771,
       -0.142541, -0.220103, -0.376406,  0.053174, -0.29507 ,  0.301268,
        0.026918,  0.19893 , -0.071954,  0.082524,  0.095058, -0.092898,
        0.044925,  0.086447,  0.304554, -0.141964, -0.100772, -0.165357,
        0.156622, -0.021282,  0.001615,  0.091452, -0.407101,  0.039256,
       -0.059249,  0.203685, -0.000725,  0.387656, -0.318588,  0.232357,
        0.183198,  0.040731,  0.177186,  0.351854, 

In [0]:
sentences = pd.read_csv('sqlResult_1558435.csv', encoding = "gb18030")['content'].to_list()

In [0]:
sentences[:3]

['此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n',
 '骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考虑性能而去屏蔽掉小核心。相反，他们正联手微软，找到一种适合桌面平台的、兼顾性能和功耗的完美方案。\r\n报道称，微软已经拿到了一些新的源码，以便Windows 10更好地理解big.little架构。\r\n资料显示，骁龙835作为一款集成了CPU、GPU、基带、蓝牙/Wi-Fi的SoC，比传统的Wintel方案可以节省至少30%的PCB空间。\r\n按计划，今年Q4，华硕、惠普、联想将首发骁龙835 Win10电脑，预计均是二合一形态的产品。\r\n当然，高通骁龙只是个开始，未来也许还能见到三星Exynos、联发科、华为麒麟、小米澎湃等进入Windows 10桌面平台。\r\n',
 '此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n至于电池缩水，可能与刘作虎所说，一加手机5要做市面最轻薄大屏旗舰的设定有关。\r\n按照目前掌握的资料，一加手机5拥有5.5寸1080P三星AMOLED显示屏、6G/8GB RAM，64GB/128GB ROM，双1600万摄像头，备货量“惊喜”。\r\n根据京东泄露的信息，一加5起售价是xx99元，应该是在2799/2899/2999中的某个。\r\n']

In [0]:
[el for el in wv_from_text.most_similar('说', topn=100) if '说' not in el[0]]

In [170]:
say_words = [
 ('告诉', 0.7580879926681519),
 ('问', 0.705342173576355),
 ('解释', 0.7009956240653992),
 ('表示', 0.7002940773963928),
 ('认为', 0.6993126273155212),
 ('回答', 0.6925622224807739),
 ('讲', 0.6917049884796143),
 ('觉得', 0.6904640197753906),
 ('直言', 0.6676661968231201),
 ('称', 0.645722508430481),
 ('提到', 0.6329718828201294),
]
say_words = set([el[0] for el in say_words]) | {'强调', '报道', '早评', '说'}
say_words.update({'表示', '说', '指出', '认为', '坦言', '告诉', '强调', '直言', '回应', '介绍', '透露', '重申', '呼吁', '感叹', '写道', '证实', '猜测', '暗示', '感慨', '指责', '主张', '表态', '承认', '提到', '引述', '抨击'})
say_words.update({})
print(say_words)

{'讲', '透露', '直言', '介绍', '问', '觉得', '强调', '说', '引述', '告诉', '指责', '回应', '证实', '感叹', '呼吁', '解释', '指出', '写道', '抨击', '回答', '感慨', '坦言', '认为', '报道', '主张', '暗示', '重申', '猜测', '早评', '表示', '表态', '称', '承认', '提到'}


In [0]:
se = sentences[12]
se

'中新网6月19日电 据外媒报道，美国底特律一名男子1976年因为一根头发被定谋杀罪，监禁41年后，终于在协助下成功洗刷罪名，于本月15日获释。\r\n据悉，61岁的沃金斯被控在1975年抢劫25岁女子伊薇特，并将其枪杀。当年20岁的沃金斯被警方逮捕，最后被以一级谋杀罪定罪，而他被定罪的关键仅是在现场的一根头发。\r\n西密西根大学库利法学院的“清白项目”为了协助更多蒙受冤狱之苦的受刑人，积极帮助沃金斯，发现当年警局实验室的分析师基于在现场找到的一根头发，而认定该案与沃金斯有关。\r\n清白专案总监马拉米契尔说，根据头发定罪不是建立在科学的基础上。该组织今年1月要求法院撤销沃金斯的定罪。\r\n沃金斯等待41年后终于真相大白，被获撤销罪名后，走出底特律下城的韦恩郡监狱。他说：“这真梦幻，令人难以置信，但我感觉很好，我早料到有这一天，但想不到要等41年那么长。”\r\n免责声明：本文仅代表作者个人观点，与环球网无关。其原创性以及文中陈述文字和内容未经本站证实，对本文以及其中全部或者部分内容、文字的真实性、完整性、及时性本站不作任何保证或承诺，请读者仅作参考，并请自行核实相关内容。\r\n'

In [0]:
se = sentence_split(se)
for i, el in enumerate(se):
  a = get_word_list(el)
  b = get_postag_list(a)
  c = (get_parser_list(a,b))
  se[i] = [a, b, c]
np.array(se)

array([[list(['中新网', '6月', '19日', '电', '据', '外媒', '报道', '，', '美国', '底特律', '一', '名', '男子', '1976年', '因为', '一', '根', '头发', '被', '定', '谋杀罪', '，', '监禁', '41', '年', '后', '，', '终于', '在', '协助', '下', '成功', '洗刷', '罪名', '，', '于', '本月', '15日', '获释', '。']),
        list(['n', 'nt', 'nt', 'n', 'p', 'n', 'v', 'wp', 'ns', 'n', 'm', 'q', 'n', 'nt', 'c', 'm', 'q', 'n', 'p', 'v', 'v', 'wp', 'v', 'm', 'q', 'nd', 'wp', 'd', 'p', 'v', 'nd', 'a', 'v', 'n', 'wp', 'p', 'nt', 'nt', 'v', 'wp']),
        list([(3, 'ADV'), (2, 'ATT'), (3, 'ADV'), (-1, 'HED'), (19, 'ADV'), (6, 'SBV'), (4, 'POB'), (4, 'WP'), (9, 'ATT'), (12, 'ATT'), (11, 'ATT'), (12, 'ATT'), (19, 'SBV'), (19, 'ADV'), (19, 'ADV'), (16, 'ATT'), (17, 'ATT'), (19, 'FOB'), (19, 'ADV'), (3, 'VOB'), (19, 'VOB'), (19, 'WP'), (25, 'ATT'), (24, 'ATT'), (22, 'CMP'), (32, 'ADV'), (25, 'WP'), (32, 'ADV'), (32, 'ADV'), (30, 'ATT'), (28, 'POB'), (32, 'ADV'), (19, 'COO'), (32, 'VOB'), (19, 'WP'), (38, 'ADV'), (37, 'ATT'), (35, 'POB'), (19, 'COO'), (3, 'WP')])],
  

In [0]:
a, b, c = se[0]
print(np.array(a))
_a = a + ['HEAD']
for i, el in enumerate(a):
  print(i, el, c[i], _a[c[i][0]])

['中新网' '6月' '19日' '电' '据' '外媒' '报道' '，' '美国' '底特律' '一' '名' '男子' '1976年'
 '因为' '一' '根' '头发' '被' '定' '谋杀罪' '，' '监禁' '41' '年' '后' '，' '终于' '在' '协助'
 '下' '成功' '洗刷' '罪名' '，' '于' '本月' '15日' '获释' '。']
0 中新网 (3, 'ADV') 电
1 6月 (2, 'ATT') 19日
2 19日 (3, 'ADV') 电
3 电 (-1, 'HED') HEAD
4 据 (19, 'ADV') 定
5 外媒 (6, 'SBV') 报道
6 报道 (4, 'POB') 据
7 ， (4, 'WP') 据
8 美国 (9, 'ATT') 底特律
9 底特律 (12, 'ATT') 男子
10 一 (11, 'ATT') 名
11 名 (12, 'ATT') 男子
12 男子 (19, 'SBV') 定
13 1976年 (19, 'ADV') 定
14 因为 (19, 'ADV') 定
15 一 (16, 'ATT') 根
16 根 (17, 'ATT') 头发
17 头发 (19, 'FOB') 定
18 被 (19, 'ADV') 定
19 定 (3, 'VOB') 电
20 谋杀罪 (19, 'VOB') 定
21 ， (19, 'WP') 定
22 监禁 (25, 'ATT') 后
23 41 (24, 'ATT') 年
24 年 (22, 'CMP') 监禁
25 后 (32, 'ADV') 洗刷
26 ， (25, 'WP') 后
27 终于 (32, 'ADV') 洗刷
28 在 (32, 'ADV') 洗刷
29 协助 (30, 'ATT') 下
30 下 (28, 'POB') 在
31 成功 (32, 'ADV') 洗刷
32 洗刷 (19, 'COO') 定
33 罪名 (32, 'VOB') 洗刷
34 ， (19, 'WP') 定
35 于 (38, 'ADV') 获释
36 本月 (37, 'ATT') 15日
37 15日 (35, 'POB') 于
38 获释 (19, 'COO') 定
39 。 (3, 'WP') 电


In [0]:
sentences[12:24]

['中新网6月19日电 据外媒报道，美国底特律一名男子1976年因为一根头发被定谋杀罪，监禁41年后，终于在协助下成功洗刷罪名，于本月15日获释。\r\n据悉，61岁的沃金斯被控在1975年抢劫25岁女子伊薇特，并将其枪杀。当年20岁的沃金斯被警方逮捕，最后被以一级谋杀罪定罪，而他被定罪的关键仅是在现场的一根头发。\r\n西密西根大学库利法学院的“清白项目”为了协助更多蒙受冤狱之苦的受刑人，积极帮助沃金斯，发现当年警局实验室的分析师基于在现场找到的一根头发，而认定该案与沃金斯有关。\r\n清白专案总监马拉米契尔说，根据头发定罪不是建立在科学的基础上。该组织今年1月要求法院撤销沃金斯的定罪。\r\n沃金斯等待41年后终于真相大白，被获撤销罪名后，走出底特律下城的韦恩郡监狱。他说：“这真梦幻，令人难以置信，但我感觉很好，我早料到有这一天，但想不到要等41年那么长。”\r\n免责声明：本文仅代表作者个人观点，与环球网无关。其原创性以及文中陈述文字和内容未经本站证实，对本文以及其中全部或者部分内容、文字的真实性、完整性、及时性本站不作任何保证或承诺，请读者仅作参考，并请自行核实相关内容。\r\n',
 '\u3000\u3000曹先生：风格转换前的阵痛\r\n\u3000\u3000今日早盘两地低开，之后一度震荡走高，领涨的仍然是上证50指数。临近午盘，大盘出现跳水，前市三大股指均告收绿。\r\n\u3000\u3000今日早评指出“创业板目前还没有回踩到位，所以今日还需要继续调整一下。今日主要是一个探底企稳的走势”。对于今天大盘继续探底我们有心理预期，但是以这种尾盘跳水的形式来展开调整，倒是出乎意料。\r\n\u3000\u3000由于跳水的原因，今天早上估计大家心情都不会太愉快，我以为这只是风格转换前的阵痛，更何况我们本就做好了今天探底的准备。已经埋伏进中报预期股的朋友可以不用理会目前大盘，有资金的建议继续逢低吸纳。\r\n\u3000\u3000今天早盘沪深两市小幅低开后，略有冲高，后回后震荡回落，沪指回踩3138年后在权重板块带动下指数企稳回升，沪指在上冲3150点后出现一波快速跳水，早盘沪深两市出现了小幅调整的格局。板块上太白粉，石墨烯，前海概念深海通等涨幅居前。宁夏板块，可燃冰，机场，民航游戏等板块调整就行。板块整体来说，部分板块开始走强，但是多数板块还是呈现整

In [0]:
split_character = {'\n': '\n||', '\\n': '\\n||', '。': '。||', '?': '?||', '!': '!||'}
def sentence_split(s):
  for k, v in split_character.items():
    s = s.replace(k, v)
  return s.split('||')


def find_subject(a, b, c, subject_index):
  subject_list = [a[subject_index]]
  while 1:
    flag = False
    for i, el in enumerate(c):
      if el[0] == subject_index and el[1] == 'ATT':
        flag = True
        subject_list.append(a[i])
        subject_index = i
    if not flag:
      return ''.join(subject_list[::-1])


def get_content(sentences):
  # print(1, sentences)
  sentences = sentence_split(sentences)
  # print(2, sentences)
  say_word_sentence_index = set()
  for say_word in say_words:
    for i, sen in enumerate(sentences):
      # print(say_word, sen, say_word in sen)
      if say_word in sen:
        say_word_sentence_index.add(i)
  sentences = [el for i, el in enumerate(sentences) if i in say_word_sentence_index]
  # print(3, sentences)

  return_list = []
  for sentence in sentences:
    # print(4, sentence)
    append_list = []
    say_index = None
    a = get_word_list(sentence)
    b = get_postag_list(a)
    c = (get_parser_list(a,b))

    #######################
    # for i, word in enumerate(a):
    #   if word in say_words:
    #     print(a)
    #     print(b)
    #     print(c)
    #     break
    #######################

    for i, word in enumerate(a):
      if word in say_words and c[i][1] in {'HED', 'POB'}:
        say_index = i
        break
    if say_index is None:
      continue
    # print(a)
    # if c[say_index][1] not in {'HED', 'POB'}:
    #   continue

    subject_index = None
    for i, tup in enumerate(c):
      if tup[0] == say_index and tup[1] == 'SBV':
        subject_index = i
        append_list.append(find_subject(a, b, c, i))
        break
      else:
        continue
    if subject_index is None:
      for i, el in enumerate(b[:say_index]):
        if el in {'nh', 'ni'}:
          subject_index = i
          append_list.append(find_subject(a, b, c, i))
      if subject_index is None:
        # print(sentence)
        # print(a)
        # print(b)
        # print(c)
        continue
    # print(subject_index, say_index)
    append_list.append(a[say_index])
    if subject_index < say_index:
      append_list.append(''.join(a[say_index + 1:]).lstrip('：，'))
    else:
      append_list.append(''.join(a[:say_index]).rstrip('：，'))

    return_list.append(append_list)

  return return_list
        

In [0]:
get_content(se)

['中新网', '6月', '19日', '电', '据', '外媒', '报道', '，', '美国', '底特律', '一', '名', '男子', '1976年', '因为', '一', '根', '头发', '被', '定', '谋杀罪', '，', '监禁', '41', '年', '后', '，', '终于', '在', '协助', '下', '成功', '洗刷', '罪名', '，', '于', '本月', '15日', '获释', '。']
['n', 'nt', 'nt', 'n', 'p', 'n', 'v', 'wp', 'ns', 'n', 'm', 'q', 'n', 'nt', 'c', 'm', 'q', 'n', 'p', 'v', 'v', 'wp', 'v', 'm', 'q', 'nd', 'wp', 'd', 'p', 'v', 'nd', 'a', 'v', 'n', 'wp', 'p', 'nt', 'nt', 'v', 'wp']
[(3, 'ADV'), (2, 'ATT'), (3, 'ADV'), (-1, 'HED'), (19, 'ADV'), (6, 'SBV'), (4, 'POB'), (4, 'WP'), (9, 'ATT'), (12, 'ATT'), (11, 'ATT'), (12, 'ATT'), (19, 'SBV'), (19, 'ADV'), (19, 'ADV'), (16, 'ATT'), (17, 'ATT'), (19, 'FOB'), (19, 'ADV'), (3, 'VOB'), (19, 'VOB'), (19, 'WP'), (25, 'ATT'), (24, 'ATT'), (22, 'CMP'), (32, 'ADV'), (25, 'WP'), (32, 'ADV'), (32, 'ADV'), (30, 'ATT'), (28, 'POB'), (32, 'ADV'), (19, 'COO'), (32, 'VOB'), (19, 'WP'), (38, 'ADV'), (37, 'ATT'), (35, 'POB'), (19, 'COO'), (3, 'WP')]
5 6 中新网6月19日电 据外媒报道，美国底特律一名男子1976年因为一根头发被定谋杀罪，监禁

[['外媒', '报道', '美国底特律一名男子1976年因为一根头发被定谋杀罪，监禁41年后，终于在协助下成功洗刷罪名，于本月15日获释。'],
 ['清白专案总监马拉米契尔', '说', '根据头发定罪不是建立在科学的基础上。'],
 ['他', '说', '“这真梦幻，令人难以置信，但我感觉很好，我早料到有这一天，但想不到要等41年那么长。'],
 ['其原创性',
  '证实',
  '对本文以及其中全部或者部分内容、文字的真实性、完整性、及时性本站不作任何保证或承诺，请读者仅作参考，并请自行核实相关内容。']]

In [207]:
se

'中新网6月19日电 据外媒报道，美国底特律一名男子1976年因为一根头发被定谋杀罪，监禁41年后，终于在协助下成功洗刷罪名，于本月15日获释。\r\n据悉，61岁的沃金斯被控在1975年抢劫25岁女子伊薇特，并将其枪杀。当年20岁的沃金斯被警方逮捕，最后被以一级谋杀罪定罪，而他被定罪的关键仅是在现场的一根头发。\r\n西密西根大学库利法学院的“清白项目”为了协助更多蒙受冤狱之苦的受刑人，积极帮助沃金斯，发现当年警局实验室的分析师基于在现场找到的一根头发，而认定该案与沃金斯有关。\r\n清白专案总监马拉米契尔说，根据头发定罪不是建立在科学的基础上。该组织今年1月要求法院撤销沃金斯的定罪。\r\n沃金斯等待41年后终于真相大白，被获撤销罪名后，走出底特律下城的韦恩郡监狱。他说：“这真梦幻，令人难以置信，但我感觉很好，我早料到有这一天，但想不到要等41年那么长。”\r\n免责声明：本文仅代表作者个人观点，与环球网无关。其原创性以及文中陈述文字和内容未经本站证实，对本文以及其中全部或者部分内容、文字的真实性、完整性、及时性本站不作任何保证或承诺，请读者仅作参考，并请自行核实相关内容。\r\n'

In [220]:
for sentence in sentences[1110:1120]:
  print(sentence)
  for el in get_content(sentence):
    print(el)
  print('##########################################')

《摆渡人》剧照主创亮相公映礼
凤凰网娱乐讯 观看热门影片，专家剖析细节，主创分享幕后——2016年12月23日，凤凰网娱乐独家制作的大型电影现场互动活动、中国电影活动首选平台“凤凰公映礼”再次举办。本次公映礼迎来由王家卫监制，张嘉佳执导，梁朝伟、金城武、陈奕迅、杨颖、张榕容、熊黛林等主演的《摆渡人》，导演张嘉佳，携主创王家卫、马苏、张榕容到场，与现场观众分享幕后趣事。
电影《摆渡人》改编自张嘉佳畅销小说《从你的全世界路过》的其中一篇，该片讲述了“金牌摆渡人”酒吧老板陈末和合伙人管春拯救情感落水者的故事，用快乐和温暖，抵抗这个世界的悲伤。
张嘉佳与马苏
36个小时跑4个城市，梁朝伟身体不适打点滴
原定要出席今晚公映礼的梁朝伟缺席，他让王家卫跟到场观众致歉，并录制了一段语言：“大家好，我没有办法来到电影现场，希望大家喜欢电影，也祝大家圣诞快乐。”
放完这段语言，王家卫对大家详细说明了梁朝伟缺席原因，他过去的36个小时里跑了4个城市，身体不适，被送进医院打点滴。
王家卫
监制王家卫：张嘉佳最大的改变，就是爱上了穿纸内裤
因为是张嘉佳与王家卫的第一次合作，张嘉佳现场回忆两人首次见面的场景，“第一次跟王导见面我们就喝点小酒吃吃火锅，那个时候就很紧张。因为王导约了见面，我想见多少年的一个偶像了，又紧张，觉得好可怕，万一看不上我把火锅掀我脸上怎么办，虽然是国际上的导演，但是一点架子没有，后来才放松了”。
张嘉佳说完，王家卫就开始吐槽他“嘴巴烂”，原来是两人第一次见面吃火锅的时候，王家卫被张嘉佳说“原来你这么胖”，吐槽完张嘉佳，王家卫又顺带“卖队友”，他爆料这部电影带给张嘉佳最大的改变就是——爱上穿纸内裤，基本上拍戏到现在每天都在换。王家卫的直爽，令现场气氛活跃了不少。
吐槽完了队友张嘉佳，王家卫还鼓励了他一番：“拍这个电影的过程里面，他是非常苦的，也是非常努力的。所以在这里我要说，作为一个新导演来说有这样一个成绩已经非常不错了。”
提及为什么找张嘉佳合作，王家卫表示，找张嘉佳是因为被他的书吸引，王家卫认为那些个都是他自己的故事，让他去写。最后王家卫询问张嘉佳这部电影谁适合当导演，自己无法导这部片子，因为这个书很多都是张嘉佳讲的故事，背景和年代都不是王家卫最熟悉的。所以他们一直在找合适的导演，但后来王家卫觉得，这部电影还是张嘉佳最适合来导演。
张嘉佳
导演张嘉佳：除了四大发明之外，

In [0]:
split_character = {'\n': '\n||', '\\n': '\\n||', '。': '。||', '?': '?||', '!': '!||'}
def sentence_split(s):
  for k, v in split_character.items():
    s = s.replace(k, v)
  return s.split('||')

def get_content(sentences):
  # print(1, sentences)
  sentences = sentence_split(sentences)
  # print(2, sentences)
  say_word_sentence_index = set()
  for say_word in say_words:
    for i, sen in enumerate(sentences):
      # print(say_word, sen, say_word in sen)
      if say_word in sen:
        say_word_sentence_index.add(i)
  sentences = [el for i, el in enumerate(sentences) if i in say_word_sentence_index]
  print(3, sentences)

  return_list = []
  for sentence in sentences:
    print(4, sentence)
    append_list = []
    say_index = None
    a = get_word_list(sentence)
    flag = False
    for i, word in enumerate(a):
      if word in say_words:
        say_index = i
        flag = True
        break
    if not flag:
      continue
    b = get_postag_list(a)
    if b[say_index] not in {'HEAD', 'POB'}:
      continue
    c = (get_parser_list(a,b))
    for i, tup in enumerate(c):
      if tup[0] == say_index and tup[1] == 'SBV':
        subject_index = i
        append_list.append(a[i])
      else:
        continue
    if subject_index < say_index:
      append_list.append(''.join(a[say_index + 1:]))
    return_list.append(append_list)

  return return_list
        